# Monitoring the Table Size in Snowflake Notebooks with Streamlit

A notebook that tracks the size of specific tables over time to help developers monitor storage growth trends. 

Here's what we're implementing to investigate the tables:
1. Retrieve the Top 100 largest tables
2. Analyze query patterns on the largest tables
3. Identify which tables are users interacting with

## 1. Retrieve the Top 100 largest tables

This query shows the top 100 largest tables, sorted by row count, including their size in GB, owners and last modification details.

In [ ]:
-- Top 100 largest tables with metrics
SELECT 
    CONCAT(TABLE_CATALOG, '.', TABLE_SCHEMA, '.', TABLE_NAME) AS FULLY_RESOLVED_TABLE_NAME,
    TABLE_OWNER,
    LAST_DDL,
    LAST_DDL_BY,
    ROW_COUNT,
    ROUND(BYTES / 1024 / 1024 / 1024, 2) AS SIZE_GB,
    LAST_ALTERED,
    CASE 
        WHEN LAST_DDL <= DATEADD(DAY, -90, CURRENT_DATE) THEN 'YES' 
        ELSE 'NO' 
    END AS LAST_ACCESSED_90DAYS
FROM SNOWFLAKE.ACCOUNT_USAGE.TABLES
WHERE DELETED IS NULL
  AND ROW_COUNT > 0
  AND LAST_ACCESSED_90DAYS = 'NO'
ORDER BY ROW_COUNT DESC
LIMIT 100;


You can now run this query in Python without any additional code -- simply use your cell name as a variable! We're going to convert our cell to a pandas DataFrame below to make it easier to work with 

In [ ]:
sql_top_tables.to_pandas()

## 2. Explore a specific table 

Let's explore one of these tables in greater detail to figure out the most common queries and who is using it most often. 

💡 **Pro tip:** You can interact with the below cell and select the fully resolved table name you want to explore more in your account!

In [ ]:
import streamlit as st

selection = st.text_input(label="Enter a fully resolved table path to explore")

Let's now pass that variable into a SQL query so we can grab query analytics on this table

In [ ]:
-- Grab most expensive queries on this table 
SELECT 
    '{{selection}}' as FULLY_RESOLVED_TABLE_NAME,
    q.QUERY_TEXT,
    q.QUERY_TYPE,
    SUM(CREDITS_USED_CLOUD_SERVICES) as CREDITS_USED,
    MAX(TOTAL_ELAPSED_TIME) as MAX_elapsed_time,
    AVG(TOTAL_ELAPSED_TIME)/1000 as AVG_EXECUTION_TIME_SEC
FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY q
WHERE START_TIME >= CURRENT_DATE - interval '90 days'
    AND query_text LIKE '%{{selection}}%'
GROUP BY ALL
ORDER BY AVG_EXECUTION_TIME_SEC DESC
LIMIT 10

In [ ]:
df = sql_most_expensive_queries_on_table.to_pandas()
st.dataframe(df,
             column_config={
                "CREDITS_USED": st.column_config.ProgressColumn(
                "CREDITS_USED",
                format="%.4f",
                min_value=df.CREDITS_USED.min(),
                max_value=df.CREDITS_USED.max(),
        ),
    },)

## 3. Find out which users most commonly query this table

Let's say we want to take our top most expensive query and turn it into a materialization. Who will be the users who are most likely to be impacted by our activities? 

To find out, we're going to grab the list of users who queried our table of interest in the last 90 days as well as the users who have executed the expensive query. We can then contact them when we make an update and tell them about improvements we made! 🎉 

-----

First, let's find out who has used our table in the last 90 days.  We already have a variable `selection` we can use, so we're plugging it into the below query: 

In [ ]:
-- Identify users who have queried selected table in last 90 days 
SELECT 
    USER_NAME, 
    COUNT(*) number_of_queries
FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY q
WHERE START_TIME >= CURRENT_DATE - interval '90 days'
    AND query_text LIKE '%{{selection}}%'
GROUP BY ALL
ORDER BY number_of_queries DESC


Now, let's say we want to materialize a specific long running query. Grab a query from the `py_visualization` cell from Section 2. 

We can now plug it into the `QUERY_TEXT` value below to find out who else would benefit from materializing this pattern. 

💡 **Pro tip:** If the query is too long, try a unique subset of the query in the box below

In [ ]:
query_selection = st.text_input(label="Enter the query text you want to look up")
st.write("**You Entered:** `" + query_selection + "`")

Sweet! Now we get a list of all the users who might have run this query, along with their total credit
consumption and query execution time over the last 90 days.

In [ ]:
SELECT 
    USER_NAME, 
    SUM(CREDITS_USED_CLOUD_SERVICES) as total_credits, 
    MAX(TOTAL_ELAPSED_TIME) as MAX_elapsed_time,
    AVG(TOTAL_ELAPSED_TIME)/1000 as AVG_EXECUTION_TIME_SEC
FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY q
WHERE START_TIME >= CURRENT_DATE - interval '90 days'
    AND query_text LIKE '%{{query_selection}}%'
GROUP BY ALL
ORDER BY total_credits DESC

## What to learn more?

- Snowflake Docs on [Account Usage](https://docs.snowflake.com/en/sql-reference/account-usage) and [QUERY_HISTORY view](https://docs.snowflake.com/en/sql-reference/account-usage/query_history)

- More about [Snowflake Notebooks](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-use-with-snowflake)

- For more inspiration on how to use Streamlit widgets in Notebooks, check out [Streamlit Docs](https://docs.streamlit.io/) and this list of what is currently supported inside [Snowflake Notebooks](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-use-with-snowflake#label-notebooks-streamlit-support)